<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Sentence_Transformers_sentence_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.3 MB/s 
     |████████████████████████████████| 4.9 MB 44.3 MB/s 
     |████████████████████████████████| 1.3 MB 63.4 MB/s 
     |████████████████████████████████| 120 kB 73.4 MB/s 
     |████████████████████████████████| 6.6 MB 45.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=fe7a4e49eb4747dcd6bd91675b7a7f33cd352b80c15b036d164f84bafcd85211
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [3]:
import scipy
from sentence_transformers import SentenceTransformer, util

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [5]:
sentence1 = 'It was a great day'
sentence2 = 'Today was awesome'

##Compute the sentence representation of the sentence pair:

In [6]:
sentence1_representation = model.encode(sentence1)
sentence2_representation = model.encode(sentence2)

In [8]:
sentence1_representation.shape

(768,)

In [9]:
sentence2_representation.shape

(768,)

In [10]:
cosine_sim = util.pytorch_cos_sim(sentence1_representation,sentence2_representation)
cosine_sim 

tensor([[0.9313]])

## Loading custom models

In [11]:
from sentence_transformers import models,SentenceTransformer
word_embedding_model = models.Transformer('albert-base-v2')

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

* In the Sentence-BERT, we use different strategies for obtaining the sentence representation, such as [CLS] tokens, mean pooling, or max pooling. 

* We set the pooling strategy we want to use for computing the sentence representation.

In [12]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                pooling_mode_mean_tokens=True,
                                pooling_mode_cls_token=False,
                                pooling_mode_max_tokens=False)

In [13]:
pooling_model

Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})

* We define our **Sentence-BERT** with the word embedding and pooling models, as shown in the following code

In [14]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

* The code will return a vector of shape 768 holding the representation of the given sentence.
* It basically computes the representation of every token in the given
sentence and returns the pooled value as the sentence representation 

In [15]:
model.encode('Transformers are awesome').shape

(768,)

## Finding a similar sentence with Sentence-BERT

In [16]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [17]:
master_dict = [
'How to cancel my order?',
'Please let me know about the cancellation policy?',
'Do you provide refund?',
'what is the estimated delivery date of the product?',
'why my order is missing?',
'how do i report the delivery of the incorrect items?'
]

In [18]:
inp_question = 'When is my product getting delivered?'

In [19]:
inp_question_representation = model.encode(inp_question,convert_to_tensor=True)

In [20]:
master_dict_representation = model.encode(master_dict,convert_to_tensor=True)

In [21]:
similarity = util.pytorch_cos_sim(inp_question_representation,master_dict_representation )

In [22]:
similarity

tensor([[0.5038, 0.5928, 0.5860, 0.8190, 0.5336, 0.6035]], device='cuda:0')

In [31]:
np.argmax(similarity.cpu().numpy()) # .cpu stores the result into cpu, then convert to numpy

3

In [33]:
print('The most similar question in the master dictionary to given input question is:',master_dict[np.argmax(similarity.cpu().numpy())])

The most similar question in the master dictionary to given input question is: what is the estimated delivery date of the product?


## Using Multilingual Model

In [34]:
from sentence_transformers import SentenceTransformer, util
import scipy
#Download and load the pre-trained multilingual model:
model = SentenceTransformer('distiluse-base-multilingual-cased')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

In [35]:
#Define the sentences
eng_sentence = 'thank you very much'
fr_sentence = 'merci beaucoup'

In [36]:
#Compute the embeddings
eng_sentence_embedding = model.encode(eng_sentence)
fr_sentence_embedding = model.encode(fr_sentence)

## Compute the similarity between the two sentence embeddings

In [37]:
similarity = util.pytorch_cos_sim(eng_sentence_embedding,fr_sentence_embedding)
print('The similarity score is:',similarity)

The similarity score is: tensor([[0.9840]])


* In this way, we can use the pre-trained multilingual models